# <font color = 'orange'> VGG16 Net

---

### <font color='blue'> Downloading zip file from the google drive.
* zip file contains the cat and dog folder which contains respective images.

In [1]:
import gdown

url = "https://drive.google.com/file/d/12jiQxJzYSYl3wnC8x5wHAhRzzJmmsCXP/view?usp=sharing"
file_id = url.split("/")[-2] # we want this id '12jiQxJzYSYl3wnC8x5wHAhRzzJmmsCXP' from url
print(file_id)

prefix = 'https://drive.google.com/uc?/export=download&id='
gdown.download(prefix+file_id, "catdog.zip")

12jiQxJzYSYl3wnC8x5wHAhRzzJmmsCXP


Downloading...
From: https://drive.google.com/uc?/export=download&id=12jiQxJzYSYl3wnC8x5wHAhRzzJmmsCXP
To: E:\PW Skills - Data Science\06_Computer_Vision\Week_22_CNN\catdog.zip
100%|█████████████████████████████████████████████████████████████████████████████| 9.09M/9.09M [00:03<00:00, 2.72MB/s]


'catdog.zip'

---

### <font color='blue'> Unzipping the downloaded zip file

In [10]:
# !unzip catdog.zip -> not working 

# solution
import zipfile

with zipfile.ZipFile('catdog.zip', 'r') as zip_ref:
    zip_ref.extractall('catdog')

---

### <font color='blue'> Loading data and Image generation

In [15]:
import tensorflow as tf

# locating directory
train_data_dir = 'catdog/train'
validation_data_dir = 'catdog/validation'

# setting samples
num_train_samples = 2000
num_validation_samples = 800

# setting epochs and batch size 
Epochs = 5
Batch_size = 16

# Image augmentation
# preprocessing - to convert all images into 224 x 224 which is accepted by the VGG16 Net
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function = tf.keras.applications.vgg16.preprocess_input)
validation_datagen = tf.keras.preprocessing.image.ImageDataGenerator(preprocessing_function = tf.keras.applications.vgg16.preprocess_input)

# intiating the generators using data generator for training and validation 
train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size = (224, 224),
    batch_size = Batch_size,
    class_mode = 'binary'
)

validation_generator = validation_datagen.flow_from_directory(
    validation_data_dir,
    target_size = (224, 224),
    batch_size = Batch_size,
    class_mode = 'binary'
)

Found 337 images belonging to 2 classes.
Found 59 images belonging to 2 classes.


---

### <font color='blue'> Model Building

In [16]:
# we don't want last layers as our output conatins different number of classes
base_model = tf.keras.applications.vgg16.VGG16(weights = 'imagenet', include_top = False, input_shape = (224, 224, 3))

# as vgg16 is trained on large data and assigned weights to filters so we don't want to retrain those layer so we will freeze them
for layer in base_model.layers:
    layer.trainable = False       #vgg16 parameters are made untrainable by freezing there parameters
    
    

# create the model 
model = tf.keras.models.Sequential()
# add base model layers
model.add(base_model)
# add custom layers
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation = 'relu'))
model.add(tf.keras.layers.Dropout(0.5)) # regularization - to reduce overfitting
model.add(tf.keras.layers.Dense(1, activation = 'sigmoid')) # 1 neuron in output layer as our out is binary i.e cat or dog 


model.summary()

58889256/58889256 [==============================] - 15s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 vgg16 (Functional)          (None, 7, 7, 512)         14714688  
                                                                 
 flatten (Flatten)           (None, 25088)             0         
                                                                 
 dense (Dense)               (None, 256)               6422784   
                                                                 
 dropout (Dropout)           (None, 256)               0         
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 21137729 (80.63 MB)
Trainable params: 6423041 (24.50 MB)
Non-trainable params: 14714688 (56.13 MB)
_________

---

### <font color='blue'> Model Compiling

In [21]:
adam = tf.keras.optimizers.Adam()

model.compile(
    loss = 'binary_crossentropy',
    optimizer = adam ,
    metrics = ['accuracy']
)

---

### <font color='blue'> Model Training

In [27]:
# here we have not giving data to the model we have connected the data generator to the model
history = model.fit(
    train_generator,
    epochs = Epochs,
    steps_per_epoch = num_train_samples // Batch_size,
    validation_data = validation_generator,
    validation_steps = num_validation_samples // Batch_size,
)

model.save('dog_cat_classifier.h5')

Epoch 1/5
125/125 [==============================] - 46s 354ms/step - loss: 1.3779 - accuracy: 0.9822 - val_loss: 3.2557 - val_accuracy: 0.9492


---

---

### <font color='blue'> Model Evaluation

In [29]:
score = model.evaluate(validation_generator)

# score give the validation loss and accuracy

accuracy = score[1] * 100
print('\nAccuracy = ',accuracy)

4/4 [==============================] - 5s 1s/step - loss: 3.2557 - accuracy: 0.9492

Accuracy =  94.91525292396545


---